Restructuring the files assuming that a folder called "train" and a file called "train.csv" exist in a directory called "CactusImages"

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
from shutil import copyfile
import keras
ORIGIN_PATH=os.getcwd()
TRAIN_PATH = os.path.abspath("aerial-cactus-identification/train/")
TEST_PATH = os.path.abspath("aerial-cactus-identification/test/")
META_PATH = os.path.abspath("aerial-cactus-identification/train.csv")
GENERATOR_PATH = os.path.abspath("data/")

Using TensorFlow backend.


In [2]:
metaData = pd.read_csv(META_PATH)
try :
    os.mkdir("data")
    os.chdir("./data/")
    os.mkdir("has_cactus_1")
    os.mkdir("has_cactus_0")
    os.chdir(ORIGIN_PATH)
except :
    pass

In [4]:
destination_1_path = os.path.join(ORIGIN_PATH,"data","has_cactus_1")
destination_0_path = os.path.join(ORIGIN_PATH,"data","has_cactus_0")

In [3]:

total = len(metaData.values)
progress=0.0
for row in metaData.values:
    progress+=1
    file = row[0]
    label = row[1]
    file_path = os.path.join(TRAIN_PATH,file)
    if label == 1:
        dest_path = os.path.join(destination_1_path,file)
        os.rename(file_path,dest_path)
    else :
        dest_path = os.path.join(destination_0_path,file)
        copyfile(file_path,dest_path)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'D:\\Y\\ML_HW2\\aerial-cactus-identification\\train\\0004be2cfeaba1c0361d39e2b000257b.jpg' -> 'D:\\Y\\ML_HW2\\data\\has_cactus_1\\0004be2cfeaba1c0361d39e2b000257b.jpg'

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [6]:
li= os.listdir(destination_0_path)

In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.3,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (224, 224),
                                                 batch_size =30,
                                                 class_mode = 'binary',
                                                 subset='training')
validation_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (224, 224),
                                                 batch_size =30,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 12251 images belonging to 2 classes.
Found 5249 images belonging to 2 classes.


In [9]:
import keras
vgg16_model = keras.applications.vgg16.VGG16()

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
from keras.models import Sequential
model = Sequential()
i = 0
for layer in vgg16_model.layers:
    i +=1
    if i < 23 :
        model.add(layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [11]:
for layer in model.layers:
    layer.trainable=False

In [12]:
from keras.layers import Dense
model.add(Dense(1,activation = 'softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [14]:
from keras.optimizers import Adam
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit_generator(train_set,steps_per_epoch=400,validation_data=validation_set, validation_steps=175,epochs=5,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
400/400 [==============================] - 5260s 13s/step - loss: 3.9843 - acc: 0.7501 - val_loss: 3.9757 - val_acc: 0.7506
Epoch 2/5
400/400 [==============================] - 5171s 13s/step - loss: 3.9796 - acc: 0.7504 - val_loss: 3.9757 - val_acc: 0.7506
Epoch 3/5
400/400 [==============================] - 5173s 13s/step - loss: 3.9587 - acc: 0.7517 - val_loss: 3.9757 - val_acc: 0.7506
Epoch 4/5
400/400 [==============================] - 5176s 13s/step - loss: 3.9997 - acc: 0.7491 - val_loss: 3.9757 - val_acc: 0.7506
Epoch 5/5
400/400 [==============================] - 5180s 13s/step - loss: 3.9576 - acc: 0.7518 - val_loss: 3.9757 - val_acc: 0.7506
